In [24]:
from domino.utils.jupyter import notebook_init
notebook_init()

In [25]:
import os.path
import re

import pandas as pd
np = pd.np

% matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

In [26]:
path = lambda filename, sep='/': os.path.join(*filename.split(sep))

This file is very malformed. <br/>
Commas are used as decimal separator, but in those cases, <br />
the fields are not delimited by the quotechar.

As a result, in almost all rows the number of columns is not right

In [27]:
columns = pd.read_csv(path('data/transports_original.csv'), encoding='unicode_escape', nrows=0)
columns = columns.iloc[:, :16].columns

In [28]:
df = pd.read_csv(path('data/transports_original.csv'), encoding='unicode_escape', header=None, skiprows=1)

In [29]:
columns

Index(['CODI_CAPA', 'CAPA_GENERICA', 'NOM_CAPA', 'ED50_COORD_X',
       'ED50_COORD_Y', 'ETRS89_COORD_X', 'ETRS89_COORD_Y', 'LONGITUD',
       'LATITUD', 'EQUIPAMENT', 'DISTRICTE', 'BARRI', 'NOM_DISTRICTE',
       'NOM_BARRI', 'ADRECA', 'TELEFON'],
      dtype='object')

In [30]:
def correct_coordinates(row):
    try:
        return pd.Series(
            {
                col: float(
                    '%s.%s' % (row.iloc[2*n], row.iloc[2*n + 1])
                )
                for n, col in enumerate(columns[3:9])
            }
        )
    except:
        return pd.Series({})
    
coords = df.iloc[:, 3:15].apply(
    correct_coordinates,
    axis=1
)

del correct_coordinates

In [31]:
coords.isnull().any(axis=1).sum(), len(coords)

(7, 660)

In [32]:
# There are some errors, but those are imposible to treat right, 
# since we don't know which of the 6 coordinates is the wrong one.
# We'll omit those
df = df[~coords.isnull().any(axis=1)].copy()
coords = coords[~coords.isnull().any(axis=1)]

In [33]:
# Add coords to df
df = df.drop(range(3, 15), axis=1)
df = pd.concat([df, coords], axis=1)
df = df[list(range(3)) + list(coords.columns) + list(range(15, 22))]
df.columns = columns

In [34]:
# Omit NOM_CAPA and sort columns
df = df[
    ['LATITUD', 'LONGITUD', 'EQUIPAMENT', 'DISTRICTE', 'BARRI', 'NOM_DISTRICTE', 'NOM_BARRI']
]

In [35]:
equipament = df.EQUIPAMENT.str.extract(
    r"""\s*(?P<type>.*)\s
    \((?P<line>.*)\)
    \s+-\s+
    (?P<name>[^(-]+)\s*
    (?:\((?P<exit>.*?)\))?
    .*?
    """,
    flags=re.VERBOSE, 
    expand=True
)

equipament.head()

,type,line,name,exit
0,METRO,L1,SANTA COLOMA,NaN
1,METRO,L1,SANTA COLOMA,NaN
2,METRO,L1,MARINA,Av. Meridiana
3,METRO,L1,FONDO,NaN
4,METRO,L1,MERCAT NOU,C. d'Antoni de Capmany


In [36]:
# Add equipment to df
df = pd.concat([df, equipament], axis=1)
df = df.drop(['EQUIPAMENT'], axis=1)

In [37]:
df.type = df.type.str.title()
df.name = df.name.str.title()

In [38]:
df = df.rename(
    columns={
        'LATITUD': 'latitude',
        'LONGITUD': 'longitude',
        'DISTRICTE': 'district_id',
        'BARRI': 'neighborhood_id',
        'NOM_DISTRICTE': 'district',
        'NOM_BARRI': 'neighborhood',
    }
)[['type', 'line', 'name', 'exit', 'latitude', 'longitude', 'district', 'neighborhood']]

In [39]:
df.neighborhood = df.neighborhood.str.title()

In [40]:
df.head()

,type,line,name,exit,latitude,longitude,district,neighborhood
0,Metro,L1,Santa Coloma,NaN,2.207988,41.451908,NaN,NaN
1,Metro,L1,Santa Coloma,NaN,2.208350,41.451604,NaN,NaN
2,Metro,L1,Marina,Av. Meridiana,2.186428,41.395580,Eixample,El Fort Pienc
3,Metro,L1,Fondo,NaN,2.217787,41.451728,NaN,NaN
4,Metro,L1,Mercat Nou,C. d'Antoni de Capmany,2.133190,41.373293,Sants-Montjuïc,Sants


In [41]:
# latitude and longitude are inverted!
df.latitude, df.longitude = df.longitude.copy(), df.latitude.copy()

In [42]:
df.head()

,type,line,name,exit,latitude,longitude,district,neighborhood
0,Metro,L1,Santa Coloma,NaN,41.451908,2.207988,NaN,NaN
1,Metro,L1,Santa Coloma,NaN,41.451604,2.208350,NaN,NaN
2,Metro,L1,Marina,Av. Meridiana,41.395580,2.186428,Eixample,El Fort Pienc
3,Metro,L1,Fondo,NaN,41.451728,2.217787,NaN,NaN
4,Metro,L1,Mercat Nou,C. d'Antoni de Capmany,41.373293,2.133190,Sants-Montjuïc,Sants


In [43]:
df.reset_index().to_csv(path('data/transports_preprocessed.csv'), index=False)